In [ ]:
# Create function for SBU takes in a list of items and returns the predictions for all them
# Evaluate to make sure that it works on both 0 and 1 and reasses the MCC
# Time per sentence for jaccard
# 


import tensorflow as tf
import pandas as pd
import numpy as np
from google.colab import drive

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

drive.mount('/content/gdrive/', force_remount=True)

Found GPU at: /device:GPU:0
Mounted at /content/gdrive/


In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
import pandas as pd
import numpy as np
import random


#This cell gets you to np arrays that contain all the sentences and the values coordinating to them

#principal sentences
df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/principal_filtered20.csv", delimiter=',', header=None, names=['number', 'sentence', 'element_name', 'element_type', 'reg_name', 'reg_type', 'reg_label'])
training_labels = []
for _ in range (len(df1['number'])):
    training_labels.append(1)
df1['training_labels'] = training_labels

values = df1.sentence.values
labels = df1.training_labels.values

values = np.delete(values,0)
labels = np.delete(labels,0)


# data principal
df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/data_filtered20.csv", delimiter=',', header=None, names=['number', 'sentence', 'element_name', 'element_type', 'reg_name', 'reg_type', 'reg_label'])
training_labels = []
for _ in range (len(df2['number'])):
    training_labels.append(1)
df2['training_labels'] = training_labels

bad = df2.sentence.values
zeros = df2.training_labels.values

bad = np.delete(bad,0)
zeros = np.delete(zeros,0)

done = False
added = 0
idx = 0

'''
values = values.tolist()
labels = labels.tolist()
bad = bad.tolist()
'''
limit = len(values)

while not done:
    if bad[idx] not in values:
        added =  added + 1
        values = np.append(values,bad[idx], axis=None)
        labels = np.append(labels,0, axis=None)
    if added == limit:
        done = True
    idx = idx + 1   

i = len(values) - 1
while i > 0:
    try:
        values[i] = values[i].replace("++++ ", "")
    except:
        None
    if str(values[i]) == 'nan':
        values = np.delete(values,i)
        labels = np.delete(labels,i)
    i = i - 1

zero = 0
one = 0
for i in range (len(labels)):
    curr = labels[i]
    if curr == 1:
        one = one + 1
    elif curr == 0:
        zero = zero + 1
    else:
        print("non-represented value")

item = []
val = []
i = len(values) - 1
while i >= 0:
    idx = random.randint(0, i)
    item.append(values[idx])
    val.append(labels[idx])
    values = np.delete(values,idx)
    labels = np.delete(labels,idx)
    i = i - 1
values = item
labels = val
print("zero is " + str(zero))
print("ones is " + str(one))
values = values[0:24000]
labels = labels[0:24000]
print("values is: " + str(len(values)))
print("done")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


zero is 60829
ones is 60841
values is: 24000
done


In [ ]:
  !pip install transformers

     |████████████████████████████████| 890kB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 29.8MB/s 
     |████████████████████████████████| 3.0MB 23.6MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e09af66796163fe335f23358bb8d0939acf90e10c432e7872dd793cc8625e8b1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer
from transformers.tokenization_utils import TensorType
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

input_ids = []
attention_masks = []

i = 0
for sent in values:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
    i = i + 1
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

#Print sentence 0, now as a list of IDs.
print('Original: ', values[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  For instance , laforin functions as a glycogen phosphatase , 9 the mitochondrial phosphatase PTPMT1 dephosphorylates phosphatidylglycerol phosphate , 10 and PIR1 , described in this paper , is specific to RNA . For instance , laforin functions as a glycogen phosphatase , 9 the mitochondrial phosphatase PTPMT1 dephosphorylates phosphatidylglycerol phosphate , 10 and PIR1 , described in this paper , is specific to RNA .
Token IDs: tensor([  101,  2005,  6013,  1010,  2474, 29278,  2378,  4972,  2004,  1037,
         1043,  2135,  3597,  6914,  6887,  2891, 21890, 18260,  1010,  1023,
         1996, 23079,  6887,  2891, 21890, 18260, 13866,  9737,  2102,  2487,
         2139,  8458,  2891,  8458, 10253, 26786,  6887,  2891, 21890,  3775,
         5149,  2140, 25643, 17119,  4747, 17344,  1010,  2184,  1998, 14255,
         2099,  2487,  1010,  2649,  1999,  2023,  3259,  1010,  2003,  3563,
         2000, 12987,  1012,   102])


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

24,000 training samples
6,000 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
print("total items is: " + str(len(values)))
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids = None, 
                                   attention_mask = b_input_mask, 
                                   labels = b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

total items is: 30000

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    750.    Elapsed: 0:00:10.
  Batch    80  of    750.    Elapsed: 0:00:19.
  Batch   120  of    750.    Elapsed: 0:00:28.
  Batch   160  of    750.    Elapsed: 0:00:37.
  Batch   200  of    750.    Elapsed: 0:00:46.
  Batch   240  of    750.    Elapsed: 0:00:55.
  Batch   280  of    750.    Elapsed: 0:01:04.
  Batch   320  of    750.    Elapsed: 0:01:13.
  Batch   360  of    750.    Elapsed: 0:01:23.
  Batch   400  of    750.    Elapsed: 0:01:32.
  Batch   440  of    750.    Elapsed: 0:01:41.
  Batch   480  of    750.    Elapsed: 0:01:50.
  Batch   520  of    750.    Elapsed: 0:01:59.
  Batch   560  of    750.    Elapsed: 0:02:08.
  Batch   600  of    750.    Elapsed: 0:02:17.
  Batch   640  of    750.    Elapsed: 0:02:27.
  Batch   680  of    750.    Elapsed: 0:02:36.
  Batch   720  of    750.    Elapsed: 0:02:45.

  Average training loss: 0.27
  Training epcoh took: 0:02:52

Running Validation...
  Ac

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/gdrive/My Drive/Colab Notebooks/30k_version_v2/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to /content/gdrive/My Drive/Colab Notebooks/30k_version_v2/


('/content/gdrive/My Drive/Colab Notebooks/30k_version_v2/vocab.txt',
 '/content/gdrive/My Drive/Colab Notebooks/30k_version_v2/special_tokens_map.json',
 '/content/gdrive/My Drive/Colab Notebooks/30k_version_v2/added_tokens.json')

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
import os
from transformers import BertForSequenceClassification, BertTokenizer

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/60k_version_v2/")
tokenizer = BertTokenizer.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/60k_version_v2/")

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers.tokenization_utils import TensorType
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from sklearn.metrics import matthews_corrcoef
import torch
import os
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import csv


pos = []
neg = []
falsePos = []
falseNeg = []


def getScores(sentences, labels, name):
  print("entered getScores")
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Set the batch size.  
  batch_size = 32  

  # Create the DataLoader.
  prediction_data = TensorDataset(input_ids, attention_masks, labels)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  # Prediction on test set

  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []
  print("before predicting")
  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  print("after predicting")
  # Combine the results across all batches. 
  flat_predictions = np.concatenate(predictions, axis=0)

  # For each sample, pick the label (0 or 1) with the higher score.
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

  # Combine the correct labels for each batch into a single list.
  flat_true_labels = np.concatenate(true_labels, axis=0)

  #print(len(flat_true_labels))

  cnt = 0
  for i in range (len(flat_predictions)):
    if flat_predictions[i] != flat_true_labels[i]:
        cnt = cnt + 1
  #print(cnt)
  print("before types")
  #print(flat_predictions)
  #print(flat_true_labels)
  df = pd.DataFrame()
  df['sentence'] = sentences
  df['prediction'] = flat_predictions
  df['label'] = flat_true_labels
  print(df)
  pos = np.array([])
  neg = np.array([])
  falsePos = np.array([])
  falseNeg = np.array([])
  #Modify this with a pandas df
  for index,row in df.iterrows():
    if row['prediction'] == 1 and row['label'] == 1:
      pos = np.append(pos,row['sentence'], axis=None)
    elif row['prediction'] == 0 and row['label'] == 0:
      neg = np.append(neg,row['sentence'], axis=None)
    elif row['prediction'] == 1 and row['label'] == 0:
      falsePos = np.append(falsePos,row['sentence'], axis=None)
    elif row['prediction'] == 0 and row['label'] == 1:
      falseNeg = np.append(falseNeg,row['sentence'], axis=None)
  text_file = open("/content/gdrive/My Drive/Colab Notebooks/" + name + "/" + name + "_pos.txt", "w")
  for elem in pos:
    text_file.write(elem)
    text_file.write("\n")
  text_file.close()
  text_file = open("/content/gdrive/My Drive/Colab Notebooks/" + name + "/" + name + "_neg.txt", "w")
  for elem in neg:
    text_file.write(elem)
    text_file.write("\n")
  text_file.close()
  text_file = open("/content/gdrive/My Drive/Colab Notebooks/" + name + "/" + name + "_falsepos.txt", "w")
  for elem in falsePos:
    text_file.write(elem)
    text_file.write("\n")
  text_file.close()
  text_file = open("/content/gdrive/My Drive/Colab Notebooks/" + name + "/" + name + "_falseneg.txt", "w")
  for elem in falseNeg:
    text_file.write(elem)
    text_file.write("\n")
  text_file.close()
  accuracy = 10
  precision = 10
  recall = 10
  f1 = 10
  '''
  print(len(pos))
  print(len(neg))
  print(len(falsePos))
  print(len(falseNeg))
  '''
  try:
    accuracy = (len(pos) + len(neg))/(len(sentences))
  except:
    print("No Accuracy")
  try:
    precision = len(pos)/(len(pos) + len(falsePos))
  except:
    print("No Precision")
  try:
    recall = len(pos)/(len(pos) + len(falseNeg))
  except:
    print("No Recall")
  try:
    f1 = (2*(recall*precision))/(recall + precision)
  except:
    print("No F1 Score")


  print("Accuracy is: " + str(accuracy))
  print("Precision is: " + str(precision))
  print("Recall is: " + str(recall))
  print("F1 Score is: " + str(f1))
  '''
  print("Positive")
  print(pos)
  print("- - - - -")
  print("Negative")
  print(neg)
  print("- - - - -")
  print("False Positive")
  print(falsePos)
  print("- - - - -")
  print("False Negative")
  print(falseNeg)
  print("- - - - -")
  '''
  print("done")

  return flat_predictions

def main(name):
  #print("- - - - - Training Data - - - - -")
  #print(getScores(values[0:48000],labels[0:48000]))
  print("- - - - - Testing Data - - - - -")
  print(getScores(values,labels,name))

if __name__ == "__main__":
  if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
  else:
      print('No GPU available, using the CPU instead.')
      device = torch.device("cpu")
  model_names = ["1k_version","10k_version", "30k_version_v2", "60k_version_v2", "100k_version", "all_version"]  
  for name in model_names:
    # Load a trained model and vocabulary that you have fine-tuned
    print("model: " + name)
    
    dir = "/content/gdrive/My Drive/Colab Notebooks/" + name + "/"
    print(name + " directory is: " + dir)
    model = BertForSequenceClassification.from_pretrained(dir)
    tokenizer = BertTokenizer.from_pretrained(dir)
    
    # Copy the model to the GPU.
    model.to(device)
    main(name)

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
model: 1k_version
1k_version directory is: /content/gdrive/My Drive/Colab Notebooks/1k_version/
- - - - - Testing Data - - - - -
entered getScores


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Predicting labels for 24,000 test sentences...
before predicting
after predicting
before types
                                                sentence  prediction  label
0      The proliferation decreased by myostatin to 54...           0      0
1      SU6556 did not inhibit EGF induced EGFR phosph...           1      1
2      Of note , KLF10 transcription in gammadelta 27...           1      1
3      Along this line , trafficking of MT1-MMP invol...           0      0
4      TGF-beta1 promotes EMT both in CSLCs and non C...           1      1
...                                                  ...         ...    ...
23995  In apoptosis , the Bcl-2 family of proteins pl...           0      1
23996  We found that our model of PDT leads to DNA fr...           1      1
23997  Unlike the MT1 subfamily , MT2A was upregulate...           1      1
23998  It was observed that in mice challenged with U...           0      0
23999  Toll like receptor 4 signal transduction stimu...           0 

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Predicting labels for 24,000 test sentences...
before predicting
after predicting
before types
                                                sentence  prediction  label
0      The proliferation decreased by myostatin to 54...           0      0
1      SU6556 did not inhibit EGF induced EGFR phosph...           1      1
2      Of note , KLF10 transcription in gammadelta 27...           1      1
3      Along this line , trafficking of MT1-MMP invol...           0      0
4      TGF-beta1 promotes EMT both in CSLCs and non C...           1      1
...                                                  ...         ...    ...
23995  In apoptosis , the Bcl-2 family of proteins pl...           1      1
23996  We found that our model of PDT leads to DNA fr...           1      1
23997  Unlike the MT1 subfamily , MT2A was upregulate...           1      1
23998  It was observed that in mice challenged with U...           0      0
23999  Toll like receptor 4 signal transduction stimu...           0 

...

Predicting labels for 24,000 test sentences...
before predicting
after predicting
before types
                                                sentence  prediction  label
0      The proliferation decreased by myostatin to 54...           1      0
1      SU6556 did not inhibit EGF induced EGFR phosph...           1      1
2      Of note , KLF10 transcription in gammadelta 27...           1      1
3      Along this line , trafficking of MT1-MMP invol...           0      0
4      TGF-beta1 promotes EMT both in CSLCs and non C...           1      1
...                                                  ...         ...    ...
23995  In apoptosis , the Bcl-2 family of proteins pl...           1      1
23996  We found that our model of PDT leads to DNA fr...           1      1
23997  Unlike the MT1 subfamily , MT2A was upregulate...           1      1
23998  It was observed that in mice challenged with U...           0      0
23999  Toll like receptor 4 signal transduction stimu...           0 

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Predicting labels for 24,000 test sentences...
before predicting
after predicting
before types
                                                sentence  prediction  label
0      The proliferation decreased by myostatin to 54...           0      0
1      SU6556 did not inhibit EGF induced EGFR phosph...           1      1
2      Of note , KLF10 transcription in gammadelta 27...           1      1
3      Along this line , trafficking of MT1-MMP invol...           0      0
4      TGF-beta1 promotes EMT both in CSLCs and non C...           1      1
...                                                  ...         ...    ...
23995  In apoptosis , the Bcl-2 family of proteins pl...           1      1
23996  We found that our model of PDT leads to DNA fr...           1      1
23997  Unlike the MT1 subfamily , MT2A was upregulate...           1      1
23998  It was observed that in mice challenged with U...           0      0
23999  Toll like receptor 4 signal transduction stimu...           0 

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Predicting labels for 24,000 test sentences...
before predicting
after predicting
before types
                                                sentence  prediction  label
0      The proliferation decreased by myostatin to 54...           0      0
1      SU6556 did not inhibit EGF induced EGFR phosph...           1      1
2      Of note , KLF10 transcription in gammadelta 27...           1      1
3      Along this line , trafficking of MT1-MMP invol...           0      0
4      TGF-beta1 promotes EMT both in CSLCs and non C...           1      1
...                                                  ...         ...    ...
23995  In apoptosis , the Bcl-2 family of proteins pl...           1      1
23996  We found that our model of PDT leads to DNA fr...           1      1
23997  Unlike the MT1 subfamily , MT2A was upregulate...           1      1
23998  It was observed that in mice challenged with U...           0      0
23999  Toll like receptor 4 signal transduction stimu...           0 